# 部分固有値と付随する固有ベクトル（部分固有対）の精度保証付き数値計算

MATLABの区間演算パッケージである[INTLAB](https://www.tuhh.de/ti3/rump/intlab/)には一般化固有値問題の固有値・固有ベクトルを精度保証付きで計算する`verifyeig`という関数が実装されている。本稿では、文献1および2を参考に`verifyeig`関数のアルゴリズムを説明し、一般固有値問題 

$$
Ax=\lambda Bx
$$

を満たす固有値 $\lambda\in\mathbb{C}$ と固有ベクトル $x\in\mathbb{C}^n$ ($n$ は行列の次元) を求める実装を紹介する。

行列 $A,B\in \mathbb{C}^{n\times n}$ を与え、$B$ は正則、$B^{-1}A$ が対角化可能、かつ全ての固有値 $\lambda$ が重複固有値でないと仮定する。写像 $F:\mathbb{C}^{n+1}\to\mathbb{C}^{n+1}$ を

$$
    F(\lambda, x) := \begin{bmatrix}x^Hx -1\\ Ax-\lambda Bx\end{bmatrix}
$$

と定義すると $F(\lambda, x)=0$ をみたす $(\lambda, x)$ は一般化固有値問題の固有対となる（固有ベクトルは $\|x\|_2=1$ と正規化されている）。いま $\bar x\in \mathbb{C}^n$, $\bar \lambda \in \mathbb{C}$ を $F(\bar\lambda,\bar x)\approx 0$ とする近似解とし、
$R\in \mathbb{C}^{n+1\times n+1}$ を写像 $F$ の近似解におけるヤコビ行列の近似逆行列（$R\approx DF(\bar\lambda,\bar x)^{-1}$）とする。部分固有値・固有ベクトルの精度保証付き数値計算には次の定理を利用する。

**定理** $\boldsymbol{y}\in \mathbb{IC}^n, \boldsymbol{d} \in \mathbb{IC}$ をある区間ベクトルと区間として、$\boldsymbol{w}:=(\boldsymbol{d},\boldsymbol{y}^T)^T\in \mathbb{IC}^{n+1}$ とする（${}^T$はベクトルの単なる転置を意味する）。 さらに

$$
    g(\boldsymbol{w}):=z+\left(I-R\cdot DF\left(\bar\lambda + \boldsymbol{d}, \bar x + \boldsymbol{y}\right)\right)\boldsymbol{w}
$$

と定義する。上の式のヤコビ行列 $DF(\lambda,x)$ と $z$ は

$$
DF(\lambda,x):=
\begin{bmatrix}
0 & 2x^T\\
-B\boldsymbol{x} & A-\boldsymbol{\lambda}B \\
\end{bmatrix},\quad
z:=-R
\left[\begin{array}
a\bar x_1^2+\bar x_2^2+\dots +\bar x_n^2-1 \\
A\bar x-\bar\lambda B\bar x \\
\end{array}\right] 
$$

で与えられる。
このとき $\mathrm{int}(\boldsymbol{w})$ を区間ベクトル $\boldsymbol{w}$ の（要素ごとの）内部とすると、$g(\boldsymbol{w})\subset \mathrm{int}(\boldsymbol{w})$ ならば、一般固有値問題 $Ax=\lambda Bx$ の真の固有値 $\lambda$ が
$\bar \lambda +\boldsymbol{d}$ 内に唯一存在し、対応する固有ベクトルが $\bar x + \boldsymbol{y}$ に包含される。

この定理をもとに精度保証付き数値計算を次のように実装する。

1. 近似固有値 $\bar{\lambda}$、近似固有ベクトル $\bar{x}$ を入力し、これらが実数であるときは実数の区間・区間ベクトルで、いずれかが複素数であるときは複素数の区間・区間ベクトルで $\boldsymbol{y}$, $\boldsymbol{d}$ を定義し、候補ベクトル $\boldsymbol{w}=(\boldsymbol{d},\boldsymbol{y}^T)^T$ を作成する。
1. ヤコビ行列$DF$ と $z$、$g(\boldsymbol{w}):=z+\left(I-R\cdot DF\left(\bar\lambda + \boldsymbol{d}, \bar x + \boldsymbol{y}\right)\right)\boldsymbol{w}$ を区間演算により計算する。
1. $g(\boldsymbol{w})\subsetneq \boldsymbol{w}$を満たすとき、$(\bar\lambda,\bar x)^T+g(\boldsymbol{w})$ の値を返す。

上記の手順に従って次元 $n=30$ のある行列 $A,B\in \mathbb{C}^{n\times n}$ に対し、$Ax=\lambda Bx$ を考える。
まず近似固有値$\bar{\lambda}$、近似固有ベクトル$\bar{x}$を計算する。

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i9-10900K CPU @ 3.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 4


In [2]:
using LinearAlgebra
n = 30
A, B = randn(n,n), randn(n,n)
λ, x = eigen(A, B)

GeneralizedEigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
30-element Vector{ComplexF64}:
  -6.800445951042474 + 0.0im
 -3.5354257648408782 - 1.728356967038503im
 -3.5354257648408782 + 1.7283569670385028im
 -1.0414617846086751 - 0.18640273721267706im
  -1.041461784608675 + 0.18640273721267706im
 -0.9259474705662194 - 1.5651922183921942im
 -0.9259474705662193 + 1.5651922183921942im
 -0.5895227577251448 - 0.5761135900175108im
 -0.5895227577251448 + 0.5761135900175107im
 -0.5358026833639461 - 4.386244195024188im
 -0.5358026833639461 + 4.3862441950241875im
 -0.3068743351976202 - 0.3834508272365941im
 -0.3068743351976202 + 0.3834508272365941im
                     ⋮
  0.2305905063504978 - 0.5093608184061266im
 0.23059050635049783 + 0.5093608184061265im
  0.3436179670069986 - 0.10621792313817505im
  0.3436179670069986 + 0.10621792313817506im
 0.47204425812533235 - 1.4075125226125946im
 0.47204425812533235 + 1.4075125226125949im
  0.8624198086581915 - 0.188579189

1. 近似固有値 $\bar{\lambda}$、近似固有ベクトル $\bar{x}$ を入力し、これらが実数であるときは実数の区間・区間ベクトルで、いずれかが複素数であるときは複素数の区間・区間ベクトルで $\boldsymbol{y}$, $\boldsymbol{d}$ を定義し、候補ベクトル $\boldsymbol{w}=(\boldsymbol{d},\boldsymbol{y}^T)^T$ を作成する。

候補ベクトルの区間幅は大きすぎたり、小さすぎたりすると定理の検証が失敗する。そのため**ある程度**小さい大きさを選ぶ必要がある。今回の実装では $\epsilon=10^{-9}$ とした。

In [3]:
using IntervalArithmetic
lam = λ[1]
x1 = x[:,1] # 対応する固有ベクトル
x = x1./sqrt(x1'*x1)
ysize = length(x)

ϵ = 1e-9 # size of candidate vector

if isreal(lam) && isreal(x)
    lam = real(lam)
    x = real(x)
    id = ϵ*interval(-1,1)
    iy =  ϵ*interval.(-ones(ysize),ones(ysize))
else
    id =Complex(interval(-ϵ,ϵ),interval(-ϵ,ϵ))
    iy = Complex.(interval(-ϵ,ϵ)*ones(ysize),interval(-ϵ,ϵ)*ones(ysize))
end
iw = [id;iy]

31-element Vector{Interval{Float64}}:
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
   ⋮
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]
 [-1.00001e-09, 1.00001e-09]

2. ヤコビ行列$DF$ と $z$、$g(\boldsymbol{w}):=z+\left(I-R\cdot DF\left(\bar\lambda + \boldsymbol{d}, \bar x + \boldsymbol{y}\right)\right)\boldsymbol{w}$ を区間演算により計算する。

In [4]:
ix = map(Interval,x)
iDF = [0 transpose(2*(x+iy)) ; -B*(x+iy) A-(lam.+id)*B]
R = inv(mid.(iDF))
z = -R*[dot(ix,ix)-1;A*ix-lam*B*ix]
gw = z+(I-R*iDF)*iw

31-element Vector{Interval{Float64}}:
 [-2.63968e-13, 2.24998e-13]
 [-9.45051e-15, 8.71733e-15]
 [-5.48651e-15, 5.13011e-15]
 [-1.27422e-14, 1.51166e-14]
 [-1.70401e-14, 1.90648e-14]
 [-3.10465e-15, 2.81724e-15]
 [-7.43464e-15, 7.85539e-15]
 [-1.0438e-14, 8.92675e-15]
 [-3.44542e-15, 3.9286e-15]
 [-3.91808e-15, 4.22219e-15]
 [-8.0929e-15, 9.79665e-15]
 [-5.66363e-15, 5.02728e-15]
 [-9.15937e-15, 9.09832e-15]
   ⋮
 [-8.92552e-15, 8.41739e-15]
 [-1.34397e-14, 1.23845e-14]
 [-5.708e-15, 6.67696e-15]
 [-1.63964e-14, 1.84501e-14]
 [-1.09402e-14, 1.23645e-14]
 [-8.75459e-15, 7.47277e-15]
 [-6.62039e-15, 6.44366e-15]
 [-5.8684e-15, 6.5665e-15]
 [-8.04379e-15, 7.46748e-15]
 [-5.17537e-15, 5.49279e-15]
 [-7.435e-15, 7.90553e-15]
 [-4.1786e-15, 4.64709e-15]

3. $g(\boldsymbol{w})\subsetneq \boldsymbol{w}$を満たすとき、$(\bar\lambda,\bar x)^T+g(\boldsymbol{w})$ の値を返す。

In [5]:
all(gw .⊂ iw)

true

In [6]:
[lam;x]+gw

31-element Vector{Interval{Float64}}:
 [-6.80045, -6.80044]
  [0.116945, 0.116946]
  [0.156488, 0.156489]
 [-0.0471603, -0.0471602]
  [0.341958, 0.341959]
 [-0.0683396, -0.0683395]
 [-0.119093, -0.119092]
 [-0.281972, -0.281971]
 [-0.238661, -0.23866]
  [0.197621, 0.197622]
 [-0.454469, -0.454468]
  [0.234518, 0.234519]
 [-0.0699391, -0.069939]
   ⋮
  [0.380726, 0.380727]
 [-0.0402252, -0.0402251]
  [0.0644114, 0.0644115]
 [-0.0301988, -0.0301987]
  [0.0806893, 0.0806894]
 [-0.15965, -0.159649]
  [0.065282, 0.0652821]
 [-0.028067, -0.0280669]
 [-0.0271603, -0.0271602]
 [-0.0396431, -0.039643]
 [-0.0108522, -0.0108521]
  [0.112245, 0.112246]

この第一成分は対象の固有値の厳密な包含である。

In [7]:
lam + gw[1]

[-6.80045, -6.80044]

この手順を`verifyeig関数`として定義する。

In [8]:
function verifyeig(A,lam,x,B = I)
x = x./sqrt(x'*x)
ysize = length(x)

ϵ = 1e-9 

if isreal(lam) && isreal(x)
    lam = real(lam)
    x = real(x)
    id = ϵ*interval.(-1,1)
    iy =  ϵ*interval.(-ones(ysize),ones(ysize))
else
    id =Complex(interval(-ϵ,ϵ),interval(-ϵ,ϵ))
    iy = Complex.(interval(-ϵ,ϵ)*ones(ysize),interval(-ϵ,ϵ)*ones(ysize))
end
    
iw = [id;iy]
# DF(w) = [0 transpose(2*(x+w[2:end])) ; -B*(x+w[2:end]) A-(lam+w[1]).*B]
DF(w) = [0 transpose(2*(x+w[2:end])) ; -B*(x+w[2:end]) A-(lam.+w[1])[1]*B]
g(w) = z+(I-R*DF(w))*w
zero =zeros(ysize+1)
R = inv(DF(zero))
# R = inv(DF([lam;x]))  
z = -R*[dot(x,x)-1;A*x-lam*B*x]
gw = g(iw)
if all(gw .⊂ iw)
    return lam .+ gw[1] 
else
    return NaN
end
end


verifyeig (generic function with 2 methods)

部分固有対の精度保証方法であるので、全固有値を精度保証するには向かない。以下のように全固有値を精度保証するには $O(n^4)$ のオーダーが必要になる。全固有値を精度保証するには、例えば、[標準固有値問題の精度保証付き数値解法](https://www.risk.tsukuba.ac.jp/~takitoshi/tutorial/verifyalleig.html)のような方法が有効である。

In [9]:
A, B = randn(n,n), randn(n,n)
λ, x = eigen(A, B)

GeneralizedEigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
30-element Vector{ComplexF64}:
  -3.0002460825065427 + 0.0im
  -1.5873573516967034 + 0.0im
  -1.4013789764054334 + 0.9279343300212451im
  -1.4013789764054332 - 0.9279343300212451im
  -0.7362326897050494 - 0.5029574504872121im
  -0.7362326897050494 + 0.5029574504872121im
  -0.5579774473635717 - 1.1464574837201806im
  -0.5579774473635716 + 1.1464574837201806im
  -0.4070550755914696 + 0.0im
 -0.24683607727112783 - 0.5092703882360076im
  -0.2468360772711278 + 0.5092703882360076im
 -0.23968591003633477 + 0.24606455086823745im
 -0.23968591003633472 - 0.24606455086823747im
                      ⋮
  0.18040559200244788 - 0.48567462795962757im
  0.18040559200244788 + 0.48567462795962757im
  0.28208644428899027 - 0.1620755558225438im
  0.28208644428899027 + 0.1620755558225438im
   0.6487102027040698 - 6.902215965378197im
   0.6487102027040698 + 6.902215965378196im
    0.876296873087635 - 0.7321852745422632im

In [10]:
# ilam = zeros(Interval,length(λ))
for i = 1:length(λ)
    @show ilam = verifyeig(A,λ[i],x[:,i],B)
end

ilam = verifyeig(A, λ[i], x[:, i], B) = [-3.00025, -3.00024]
ilam = verifyeig(A, λ[i], x[:, i], B) = [-1.58736, -1.58735]
ilam = verifyeig(A, λ[i], x[:, i], B) = [-1.40138, -1.40137] + [0.927934, 0.927935]im
ilam = verifyeig(A, λ[i], x[:, i], B) = [-1.40138, -1.40137] + [-0.927935, -0.927934]im
ilam = verifyeig(A, λ[i], x[:, i], B) = [-0.736233, -0.736232] + [-0.502958, -0.502957]im
ilam = verifyeig(A, λ[i], x[:, i], B) = [-0.736233, -0.736232] + [0.502957, 0.502958]im
ilam = verifyeig(A, λ[i], x[:, i], B) = [-0.557978, -0.557977] + [-1.14646, -1.14645]im
ilam = verifyeig(A, λ[i], x[:, i], B) = [-0.557978, -0.557977] + [1.14645, 1.14646]im
ilam = verifyeig(A, λ[i], x[:, i], B) = [-0.407056, -0.407055]
ilam = verifyeig(A, λ[i], x[:, i], B) = [-0.246837, -0.246836] + [-0.509271, -0.50927]im
ilam = verifyeig(A, λ[i], x[:, i], B) = [-0.246837, -0.246836] + [0.50927, 0.509271]im
ilam = verifyeig(A, λ[i], x[:, i], B) = [-0.239686, -0.239685] + [0.246064, 0.246065]im
ilam = verifyeig(A, λ[i],

**TODO**　固有値が縮退し多重固有値である場合や複数の固有値がクラスター形成している場合、この方法は失敗する。この場合の精度保証付き数値計算方法は文献3にある。今後この方法を実装していきたい。


本資料は以下のような文献・Web ページ等を参考に書いている。

### 参考文献

1. 大石進一編著, 精度保証付き数値計算の基礎, コロナ社, 2018.<br>
（精度保証付き数値計算の教科書、3.4.2章にある「非線形方程式を利用した精度保証法」を実装した）

1. S.M. Rump. Guaranteed Inclusions for the Complex Generalized Eigenproblem. Computing, 42:225-238, 1989.<br>
（今回の方法を初めて発表した原著論文）

1. S.M. Rump. Computational Error Bounds for Multiple or Nearly Multiple Eigenvalues. Linear Algebra and its Applications (LAA), 324:209–226, 2001.<br>
（多重固有値や多数の固有値のクラスターに対する精度保証付き数値計算方法。INTLABの`verifyeig.m`にはこの方法も実装されている）


<div align="right">近藤慎佑, <a href="http://www.risk.tsukuba.ac.jp/~takitoshi/">高安亮紀</a>，2022年9月19日</div>